<a href="https://colab.research.google.com/github/witusj/obp/blob/master/outpatient_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Import packages
import numpy as np
import math
import random
from scipy.stats import poisson
from scipy.special import binom

## Setup problem definition

## Test case

- T = 4h
- d = 5 min
- beta = 9 minutes service time (exponential)
- N = 24 patients

In [ ]:
class schedule:
  def __init__(self, beta: int, T: int, d: int, N: int):
    self.beta = beta # = 1/μ : average service time
    self.T = T # number of intervals
    self.d = d # length of interval
    self.N = N # total number of patients
    self.x:list[int] = np.repeat(0, self.T) # schedule with x[t] as number of 
                                            # patients scheduled at the start of
                                            # interval t, t = 1,...,T; initially
                                            # set at zero
  def reset_schedule(self):
    for t in range(self.T):
      self.x[t] = 0
  def make_random_schedule(self, min_x: int, max_x: int, step: int=1):
    n = self.N
    for i in range(0,self.T, step):
      if n >= 0:
        r = random.randint(min_x, max_x)
        self.x[i] = min(r,n)
        n = n - r
        print(n, self.x[i])
      else:
        return
  def make_initial_schedule(self):
    for i in range(self.N):
      t = round(i*self.T / self.N)
      print(i)
      if i > self.T:
        i-= 1
      self.x[t] += 1

beta = 20
T = 48
d = 5
N = 10
s1 = schedule(beta,T,d,N)
step = math.floor(beta/d)
s1.make_random_schedule(0,2, step)
print(s1.x)
print(f'total number of patients in schedule {sum(s1.x)}')
s1.reset_schedule()
print(s1.x)
s1.make_initial_schedule()
print(s1.x)
print(f'total number of patients in schedule {sum(s1.x)}')

9 1
9 0
9 0
9 0
9 0
9 0
8 1
8 0
6 2
5 1
4 1
2 2
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 2 0 0 0 1
 0 0 0 1 0 0 0 2 0 0 0]
total number of patients in schedule 8
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
0
1
2
3
4
5
6
7
8
9
[1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0
 0 1 0 0 0 0 1 0 0 0 0]
total number of patients in schedule 10


## Functions

In [ ]:
def recursive_sum(i):
  if i==1:
    return 1
  return i+recursive_sum(i-1)

recursive_sum(5)

15

In [ ]:
#not used because of recursion depth limit in Python
#def factorial(i):
#  if i==1:
#    return 1
#  return i * factorial(i-1)
#
#factorial(5)

for i in range(10):
  print(math.factorial(i))

1
1
2
6
24
120
720
5040
40320
362880


In [ ]:
# Not necessary - use scipy poisson.pmf instead
def service(k, beta):
  return pow(beta,k)*math.exp(-beta) / math.factorial(k)

service(10,20)

0.005816306518345136

In [ ]:
# Not necessary - use scipy binom instead
def binomial_coef(n,k):
  return factorial(n)/(factorial(k)*factorial(n-k))

print(f'{binomial_coef(4,1)} {binomial_coef(4,2)} {binomial_coef(4,3)}')

4.0 6.0 4.0


In [ ]:
print(f'{binom(4,1)} {binom(4,2)} {binom(4,3)}')

4.0 6.0 4.0


In [ ]:
# moving patients iteratively, local search
def iteration_small(x, results, I, n): # name of function should be meaningful; add types.
  x_new = x.copy()
  for m in range(1, I-1):
    for k in list(range(1, I))[::-1]:
      if x_new[k] > 0:
        x_new[k] -= 1
        x_new[((k + m - 1) % I) + 1] += 1
        results_temp = results(x_new, 0) # class for storing results?
        if results_temp['objVal'] < results['objVal']: # need to use eval method instead?
          return x_new, results_temp
        else: 
          x_new[k] += 1
          x_new[((k + m - 1) % I) + 1] -= 1 # store index as variable?
  return X, None

In [ ]:
def limit(mu): # name of function should be meaningful
  return max(mu+4*math.sqrt(mu),100)

limit(1/beta)

100

In [ ]:
def create_a(beta, precision=0.9999): #Poisson P(potential departures = k)
  k = 0
  a=[]
  while sum(a) < precision:
    a.append(service(k, beta)) # Service function not necessary - use scipy poisson instead
    k+=1
  return(a)

a1=create_a(10)
a1

[4.5399929762484854e-05,
 0.00045399929762484856,
 0.0022699964881242427,
 0.007566654960414142,
 0.018916637401035354,
 0.03783327480207071,
 0.06305545800345118,
 0.09007922571921599,
 0.11259903214901998,
 0.1251100357211333,
 0.1251100357211333,
 0.11373639611012118,
 0.09478033009176766,
 0.07290794622443666,
 0.05207710444602619,
 0.03471806963068413,
 0.021698793519177577,
 0.012763996187751515,
 0.007091108993195286,
 0.0037321626279975192,
 0.0018660813139987594,
 0.0008886101495232189,
 0.00040391370432873584,
 0.00017561465405597208,
 7.317277252332172e-05]

In [ ]:
def create_a(beta, precision=0.9999): #Poisson P(potential departures = k)
  k = 0
  a=[]
  while sum(a) < precision:
    a.append(poisson.pmf(k=k, mu=beta))
    k+=1
  return(a)

a2=create_a(10)
a2

[4.5399929762484854e-05,
 0.0004539992976248486,
 0.0022699964881242435,
 0.007566654960414144,
 0.01891663740103538,
 0.03783327480207079,
 0.06305545800345125,
 0.090079225719216,
 0.11259903214902009,
 0.12511003572113372,
 0.12511003572113372,
 0.11373639611012128,
 0.09478033009176803,
 0.07290794622443707,
 0.05207710444602615,
 0.034718069630684245,
 0.021698793519177594,
 0.012763996187751505,
 0.007091108993195334,
 0.003732162627997529,
 0.0018660813139987742,
 0.0008886101495232241,
 0.0004039137043287357,
 0.00017561465405597286,
 7.317277252332212e-05]

In [ ]:
def calcExponentialLimit(mu):
  return max(mu+4*math.sqrt(mu),100)

In [ ]:
def binomCoeff(k, i):
  math.factorial(k) / (math.factorial(k - i) * math.factorial(i))

def binomPMF(k, i, m, add_v, no_show):
  binomCoeff(k, m) * add_v[m][i] * (1 - no_show)^m * no_show^(k-m)

In [ ]:
# two main components
# find a schedule to evaluate <- efficient search procedure to still end up with a good schedule
# evaluate a specific schedule <- as fast a possible
def calculateProbabilities(x, p, beta, precision, n, n_temp, no_show, I):
  count = len(p)
  waiting = 0
  idletime = 0
  tardiness = 0
  p_plus = []
  p_min = []
  v = []
  w = []
  add_v = [] # needs a predetermined size

#	v[k][i]=probability of having i units of work given that 
#	k patients are scheduled for the interval.
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.

  for k in range(n_temp):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_k = 0
    while sum_k < precision and i <= limit:
      if k == 0:
        if i > 0:
          add_v[k][i] = 0
        else if i == 0:
          add_v[k][i] = 1
      else:
        n = 0
        while n <= count:
          add_v[k][i] += p[n] * add_v[k-1][i-n]
          n++
      sum_k += add_v[k][i]
      i++
  for k in range(n_temp):
    i = 0
    sum_k = 0
    while sum_k < precision and i <= limit:
      for m in range(k):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
	    sum_k += v[k][i]
	    i++
  
  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    if i == 0:
      p_min[1][i] = 1
    else: 
      p_min[1][i] = 0
    p_plus[1][i] = v[x[1]][i] * p_min[1][0]
    sum_p += p_plus[1][i]
    i++
  for t in range(2, I+1):
    if not x[t]:
      x[t] = 0
    # Constraint 3
    for k in range(d):
      p_min[t]][0] += p_plus[t-1][k]
    # Constraint 4
    for i in range(1,limit):
      p_min[t][i] = p_plus[t-1][i+d]
    # Constraint 5
    if t != I+1:

	    